## Impressions and Reviews  
- lasso 계수를 통해 중요변수 도출
- hyperopt 에서 lgb train을 쓸 때 굳이 따로 kfold를 사용한이유는? (lgb.cv 대신)

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/content/"))

# Any results you write to the current directory are saved as output.

['.config', 'train.csv.zip', 'train.csv', 'test.csv.zip', 'test.csv', 'sample_submission.csv.zip', 'sample_data']


In [7]:
import os
import gc
import time
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
import lightgbm as lgb
# from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval 
# from hyperopt.pyll.base import scope

In [13]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
train['log_target'] = np.log1p(train.target)
print(train.shape, test.shape)

(4459, 4994) (49342, 4992)


In [14]:
train.head(2)

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,...,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466,log_target
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,1300000.0,...,0,1100000.0,0,0,0,0.0,0.0,14800000,0.0,1200000.0,0.0,0.0,0,0,0,0,0.0,4000000,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,17.453097
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,13.304687


All the given features are numeric and no categorical. All the features are anonymized.                 
There are approx 5000 features present and so feature selection becomes a very important exercise.            

In [15]:
X_cols = [col for col in train.columns if col not in ['ID','target','log_target']]
print(len(X_cols))

4991


In [16]:
# max -min >1000 이면 log1p 적용
# Converting features into log transformation
feats_to_convert = []
for col in X_cols:
    diff = train[col].max() - train[col].min()
    if diff>1000:
        feats_to_convert.append(col)
print(len(feats_to_convert))

train[feats_to_convert] = np.log1p(train[feats_to_convert].values)
test[feats_to_convert] = np.log1p(test[feats_to_convert].values)

4735


Lasso for Feature Selection          
**TODO** Finding best alpha value by Grid-Search based tuning or Hyperopt

In [17]:
# lasso 계수를 통해 중요변수 도출
lasso_mod = Lasso(alpha=0.05, max_iter=1000, fit_intercept=True, normalize=False, random_state=42)
lasso_mod.fit(X=train[X_cols].values, y=train.log_target.values)
imp_feats_indexes = np.nonzero(lasso_mod.coef_)[0]
print(imp_feats_indexes)
imp_feats = np.array(X_cols)[imp_feats_indexes]
print('Number of important features selected by lasso:', len(imp_feats))
print('Important features are:', imp_feats)

[   5    8   14   22   29   32   37   40   49   53   61   90  108  118
  133  137  166  168  185  203  207  216  217  220  225  237  240  261
  267  271  272  274  278  285  291  295  303  306  309  314  319  335
  338  340  350  353  355  359  381  387  392  406  408  427  435  437
  438  442  444  451  458  491  498  499  500  501  502  504  518  519
  539  552  561  572  574  578  606  607  608  613  628  639  653  655
  671  687  693  706  709  713  719  726  732  755  756  760  763  765
  774  783  784  794  798  809  815  816  817  830  833  839  847  851
  858  859  866  872  873  880  887  902  905  913  916  917  922  923
  925  928  937  944  948  955  959  962  968  987  989  998  999 1006
 1012 1016 1030 1042 1052 1057 1058 1059 1062 1090 1098 1109 1119 1121
 1127 1130 1132 1140 1142 1148 1151 1155 1162 1165 1178 1185 1186 1202
 1210 1220 1224 1227 1233 1241 1248 1254 1255 1271 1276 1284 1290 1303
 1305 1309 1313 1325 1337 1363 1388 1423 1427 1434 1442 1447 1455 1463
 1478 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 48.65227739403599, tolerance: 1.3667402423569335
  positive)


Using HyperOpt for Hyper-Parameter Selection

In [ ]:
def model_metrics(y_test, preds, scores):
    scores['rmsle'].append(np.sqrt(mean_squared_error(y_test, preds)))
    #scores['mae'].append(mean_absolute_error(y_test, preds))
    return scores

def get_space(clf_choice):
    if clf_choice=='LGB':
      # 탐색 hyperparameter 범위 지정
        lgb_space ={'num_leaves': scope.int(hp.quniform('num_leaves', 50, 200, 1)),
                    'learning_rate': 0.1, #hp.uniform('learning_rate', 0.02, 0.05),
                    'max_bin': scope.int(hp.quniform('max_bin', 300, 500, 1)),
                    'num_boost_round': scope.int(hp.quniform('num_boost_round', 100, 2000, 1)),
                    'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
                    'min_child_samples': scope.int(hp.quniform('min_child_samples', 1, 100, 1)), 
                    'subsample': hp.uniform('subsample', 0.5, 1.0), 
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)}
        return lgb_space
    
def lgb_train(space, X_train, y_train, X_test):
    lgb_params ={'task':'train', 'boosting_type':'gbdt', 'objective':'regression', 'metric': {'rmse'},
                 'num_leaves': space['num_leaves'], 'learning_rate': space['learning_rate'], 'max_bin': space['max_bin'], 
                 'max_depth': space['max_depth'], 'min_child_samples':space['min_child_samples'], 'subsample': space['subsample'],
                 'colsample_bytree': space['colsample_bytree'], 'nthread':4, 'verbose': 0}
    lgbtrain = lgb.Dataset(X_train, label=y_train)
    lgbtrain.construct()
    lgb_model = lgb.train(lgb_params, lgbtrain, num_boost_round=space['num_boost_round'])
    preds = lgb_model.predict(X_test, num_iteration=space['num_boost_round'])
    return lgb_model, preds

def hyperopt_param_tuning(space, kf, clf_choice, trainX, trainY, max_evals):
    
    def objective(space):
        print('Space:', space)
        scores = {'rmsle':[]} #, 'mae':[]}
        for train_index, test_index in kf.split(trainX):
            X_train, X_test, y_train, y_test = trainX[train_index], trainX[test_index], trainY[train_index], trainY[test_index]
            lgb_model, preds = lgb_train(space, X_train, y_train, X_test)
            scores = model_metrics(y_test, preds, scores)
            print('scores', scores)
        loss = np.array(scores['rmsle']).mean()
        print('RMSLE:', loss, '\n\n')
        return{'loss':loss, 'status': STATUS_OK, 'scores':scores}
    
    trials = Trials()
    # fmin - 목적함수 최소화
    # Run the hyperparameter search
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    # Get the values of the optimal parameters
    best_params = space_eval(space, best)
    return best_params, trials


In [ ]:
trainX = train[imp_feats].values
trainY = train['log_target'].values
# 5-fold CV
kf = KFold(n_splits=10, random_state=42)    
lgb_space = get_space('LGB')
# Hyperparameter tuning
lgb_best_params, trials = hyperopt_param_tuning(lgb_space, kf, 'LGB', trainX, trainY, 100)
print('Best LGB params for our task:', lgb_best_params)
#print(trials)

In [ ]:
# Train on whole data using best params that we got from HyperOpt and lower learning_rate     
testX = test[imp_feats].values
#lgb_best_params['learning_rate'] = 0.001
#lgb_best_params['num_boost_round'] = lgb_best_params['num_boost_round'] * 80 
lgb_model, preds = lgb_train(lgb_best_params, trainX, trainY, testX)
sub = test[['ID']]
sub['target'] = np.expm1(preds)
print(sub.head(), '\n')
print(sub.describe())
sub.to_csv('first_model_prediction.csv', index=False)

**TODO**          
* More Feature Selection method like Boruta or Correaltion based FS.         
* Create feature interactions of some important features.                